In [1]:
import pyxdf
import mne
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
# FILL IN HERE
subject_nr = "P001" # Format #P001
session_nr = "S001" # Format #S001
run_nr     = "001"  # 001 / 002

In [4]:
#Reading files
f_name = f'Data/sub-{subject_nr}/ses-{session_nr}/eeg/sub-{subject_nr}_ses-{session_nr}_task-Default_run-{run_nr}_eeg'
f_format = '.xdf'
fname = f_name + f_format
streams, header = pyxdf.load_xdf(fname)

In [5]:
# Reading EEG
def configure_eeg_data(index):   
    eeg_data = streams[index]['time_series'].T
    eeg_data *= 1e-6 # uV -> V
    time_stamps_eeg = streams[index]["time_stamps"] - streams[index]['time_stamps'][0]
    sfreq_eeg = float(streams[index]['info']['nominal_srate'][0])

    time_stamps_eeg = time_stamps_eeg.reshape((1,time_stamps_eeg.shape[0]))
    time_stamps_eeg = np.repeat(time_stamps_eeg, 8, axis=0)

    return eeg_data, time_stamps_eeg, sfreq_eeg


In [6]:
# Reading Markers
#'0' marker is standard and '1' marker is oddball/target
def configure_markers(index):
    markers = streams[index]["time_series"]
    time_stamps = streams[index]["time_stamps"] - streams[index]["time_stamps"][0]

    return markers, time_stamps

In [8]:
#Reading pcg 
def configure_pcg_data(index):
    pcg_data = streams[index]["time_series"].T
    sfreq_pcg = float(streams[index]["info"]["nominal_srate"][0])

    if len(streams[index]["time_stamps"]) == 0:
        print('ERROR: Found no time stamps')
        time_stamps_pcg = []
    else:
        time_stamps_pcg = streams[index]["time_stamps"] - streams[index]["time_stamps"][1]
        plt.plot(pcg_data.T.ravel()/np.max(abs(pcg_data)))    
    
    return pcg_data, time_stamps_pcg, sfreq_pcg

In [9]:
#Directing each stream into correct function
for i in range(len(streams)):
    print(i)
    if streams[i]['info']['type'] == ['Audio']:
        print('Found pcg data')
        pcg_data, time_stamps_pcg, sfreq_pcg = configure_pcg_data(i)
    elif streams[i]['info']['type'] == ['ExG']:
        print('Found eeg data')
        eeg_data, time_stamps_eeg, sfreq_eeg = configure_eeg_data(i)
    elif streams[i]['info']['type'] == ['Markers']:
        print('Found markers')
        markers, time_stamps_markers = configure_markers(i)
    else:
        print('Found nothing')

0
Found eeg data
1
Found pcg data


In [10]:
# Creating the raw data
info = mne.create_info(8, sfreq_eeg, ch_types= "eeg", verbose=None)
print(info)
raw = mne.io.RawArray(eeg_data, info)

<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>
Creating RawArray with float64 data, n_channels=8, n_times=75072
    Range : 0 ... 75071 =      0.000 ...   300.284 secs
Ready.


In [11]:
# Renaming the channels and dropping the non-used channels
mapping = {'0':'F4','1':'Fp2','2':'C3',
           '3':'FC6','4':'O1','5':'Oz','6':'FT9', 
           '7':'T8'}
mne.rename_channels(raw.info, mapping)
#raw.drop_channels(['12','13','14','15'])

In [13]:
o_format = '.fif'
fname = f_name + o_format
raw.save(fname, overwrite=True)

Overwriting existing file.
Writing c:\Users\annej\OneDrive\Documents\GitHub\MASTER-eeg-stress-det\Data\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-Default_run-001_eeg.fif
Closing c:\Users\annej\OneDrive\Documents\GitHub\MASTER-eeg-stress-det\Data\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-Default_run-001_eeg.fif
[done]


#### Reading .fif file, event extraction and plotting raw EEG

In [14]:
 # Converted data
o_format = '.fif'
fname = f_name + o_format
raw = mne.io.read_raw_fif(fname)

Opening raw data file Data/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.fif...
Isotrak not found
    Range : 0 ... 75071 =      0.000 ...   300.284 secs
Ready.


In [15]:
#raw.plot(scalings=dict(eeg=50e-6),duration=10)

raw.plot(duration=60, title='test', n_channels=8,show_scrollbars=False)

Using qt as 2D backend.


Channels marked as bad:
none
